# Feladat: Genetikus Algoritmus - Rendszámok Elrejtése
Egy ország új rendszámformátumot szeretne bevezetni, amely 7 karakterből áll: az első három pozíció betű (A-Z), az utolsó négy pozíció szám (0-9). Például egy ilyen rendszám lehet: ABC1234.

A cél az, hogy olyan rendszámokat generáljunk, amelyek:
  1.	Minél könnyebben megjegyezhetők (például, ha tartalmaz ismétlődő karaktereket).
  2.	Specifikus mintázatot követnek, például az utolsó számjegy 4 vagy 7.
  A genetikus algoritmust használjuk arra, hogy a rendszámokat generáljuk és finomítsuk a megadott cél érdekében.

Célok:
  1.	Fitness függvény: Minden egyes rendszámot a következő kritériumok alapján értékelj:
    *	Minél több ismétlődő karakter szerepel a rendszámban, annál jobb.
    *	Az utolsó számjegy 4 vagy 7 legyen.
    *	Ha az első három betű AAA, BBB vagy CCC, akkor + bónusz pont.
  2.	Keresztezés és mutáció:
    *	Kombinálj két rendszámot, hogy újakat hozz létre.
    *	Véletlenszerűen változtass meg egy karaktert mutációval.
  3.	Populáció: Kezdj véletlenszerű rendszámokkal, és futtasd a genetikus algoritmust, amíg egy megfelelő rendszámot nem találunk.



In [351]:
!pip install pygad

In [352]:
import random
import string
import pygad
import numpy as np

In [353]:
# 1. Fitness függvény
def fitness(ga_instance, solution, solution_idx):
    license_plate = "".join(map(chr, solution))

    fitness = 0
    char_count = {char: license_plate.count(char) for char in set(license_plate)}
    for count in char_count.values():
        if count > 1:
            fitness += (count - 1) ** 2  # Ismétlődés megjutalmazása

    if license_plate[-1] in ["4", "7"]:
        fitness += 15
    else:
        fitness -= 5

    if license_plate[:3] in ["AAA", "BBB", "CCC"]:
        fitness += 20

    return fitness


In [354]:
# 2. Keresztezés és mutáció
def crossover(parents, offspring_size, ga_instance):
    offspring = []
    num_parents = len(parents)

    for k in range(offspring_size[0]):
        parent1_idx, parent2_idx = random.sample(range(num_parents), 2)
        parent1 = parents[parent1_idx]
        parent2 = parents[parent2_idx]

        crossover_point = random.randint(1, len(parent1) - 1)
        child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        offspring.append(child)

    return np.array(offspring)

def mutation(offspring, ga_instance):
    for child in offspring:
        if random.random() < 0.2:
            idx = random.randint(0, len(child) - 1)
            if idx < 3:
                child[idx] = random.randint(65, 90)  # ASCII A-Z
            else:
                child[idx] = random.randint(48, 57)  # ASCII 0-9
    return offspring


In [355]:
# 3. Populáció
def license_plate_generator():
    prefix = [random.randint(65, 90) for _ in range(3)]
    suffix = [random.randint(48, 57) for _ in range(4)]
    return prefix + suffix

In [356]:
# Kezdeti populáció
population = [license_plate_generator() for _ in range(200)]

In [358]:
# GA konfiguráció
ga_instance = pygad.GA(
    num_generations=500,
    num_parents_mating=30,
    fitness_func=fitness,
    sol_per_pop=300,
    num_genes=7,
    gene_type=int,
    gene_space=[
        {'low': 65, 'high': 90},  # A-Z ASCII karakterek
        {'low': 65, 'high': 90},
        {'low': 65, 'high': 90},
        {'low': 48, 'high': 57},  # 0-9 ASCII karakterek
        {'low': 48, 'high': 57},
        {'low': 48, 'high': 57},
        {'low': 48, 'high': 57}
    ],
    parent_selection_type="tournament",
    crossover_type=crossover,
    mutation_type=mutation,
    mutation_percent_genes=30,
    stop_criteria=["saturate_100"],
    random_seed=None
)
# Futás
ga_instance.run()

# Legjobb megoldás
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(f"Best license plate: {''.join(map(chr, solution))}, Fitness: {solution_fitness}")

Best license plate: EEE7777, Fitness: 28
